# Streamlined Extraction of Nucleic Acids and Metabolites from Low- and High-Biomass Samples Using Isopropanol and Matrix Tubes

## Set up notebook environment
### NOTE: Use a QIIME2 kernel

In [2]:
import os
import biom
import warnings
import pickle
import numpy as np
import pandas as pd
import qiime2 as q2
from biom import Table
from skbio import OrdinationResults
from skbio.stats import subsample_counts
from skbio.stats.distance import permanova, anosim, mantel
from skbio.stats.distance import DistanceMatrix
from qiime2.plugins.gemelli.actions import rpca
from qiime2.plugins.gemelli.actions import phylogenetic_rpca_with_taxonomy
from qiime2.plugins.feature_table.actions import rarefy
from qiime2.plugins.diversity.actions import beta_group_significance
from qiime2.plugins.emperor.actions import biplot, plot
from qiime2.plugins.diversity.actions import (beta,
                                              beta_phylogenetic,
                                              pcoa)
from qiime2.plugins import demux, deblur, quality_filter, \
                           metadata, feature_table, alignment, \
                           phylogeny, diversity, emperor, feature_classifier, \
                           taxa, composition

from assets.step_wise_anova import run_stepwise_anova
from qiime2.plugins.fragment_insertion.actions import filter_features
warnings.filterwarnings("ignore", category=DeprecationWarning)

# helper functions
from assets.util_updated_again import (mantel_matched, simulate_depth,
                        all_dists, all_dists_no_tree, nested_permanova)

# plotting
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
%matplotlib inline


# Subset metadata to make paired files between extraction kits

In [ ]:
# Read in sample metadata
md = pd.read_csv('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/sample_metadata/12201_metadata.txt',
                sep = '\t')


In [ ]:
# Subset sample metadata to make files for round 1 and round 2
md_round1and2 = md[md['round'] != 3]
md_round1 = md_round1and2[md_round1and2['round'] == 1]
md_round2 = md_round1and2[md_round1and2['round'] == 2]


In [ ]:
# Subset round-specific metadata files to make files for each kit
md_round1_powersoil = md_round1[md_round1['extraction_kit'] == 'PowerSoil']
md_round1_powersoil_pro = md_round1[md_round1['extraction_kit'] == 'PowerSoil Pro']
md_round1_norgen = md_round1[md_round1['extraction_kit'] == 'Norgen']
md_round2_powersoil = md_round2[md_round2['extraction_kit'] == 'PowerSoil']
md_round2_magmax = md_round2[md_round2['extraction_kit'] == 'MagMAX Microbiome']
md_round2_nucleomag = md_round2[md_round2['extraction_kit'] == 'NucleoMag Food']
md_round2_zymo = md_round2[md_round2['extraction_kit'] == 'Zymo MagBead']


In [ ]:
# Merge kit-specific files to make paired files for comparison
md_bothPS = pd.concat([md_round1_powersoil, 
                       md_round1_powersoil_pro,
                       md_round1_norgen,
                       md_round2_powersoil,
                       md_round2_magmax,
                       md_round2_nucleomag,
                       md_round2_zymo])


In [ ]:
# Export paired files
md_bothPS.to_csv('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/sample_metadata/01_paired_files/12201_metadata_round1and2_bothPS.txt',
                           sep = '\t',
                           index = False)


# Stepwise RDA

## 16S data

In [2]:
# Import data
md_round1and2_bothPS_q2 = q2.Metadata.load('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/sample_metadata/01_paired_files/12201_metadata_round1and2_bothPS.txt')

table_16S = q2.Artifact.load('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/16S/10_filtered_data/dna_bothPS_16S_deblur_biom_lod_noChl_noMit_sepp_gg_noNTCs_noMock.qza')

tree_16S = q2.Artifact.load('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/16S/09_fragment_insertion/dna_all_16S_deblur_seqs_noChl_noMit_tree_gg.qza')


In [3]:
# Filter table
table_16S_biom = table_16S.view(Table)
md_round1and2_bothPS_df = md_round1and2_bothPS_q2.to_dataframe()
shared_ = list(set(table_16S_biom.ids()) & set(md_round1and2_bothPS_df.index))
md_round1and2_bothPS_df_16S = md_round1and2_bothPS_df.reindex(shared_)
table_16S_biom_bothPS = table_16S_biom.filter(shared_)
keep_ = table_16S_biom_bothPS.ids('observation')[table_16S_biom_bothPS.sum('observation') > 0]
table_16S_biom_bothPS.filter(keep_, axis='observation')

# Import filtered table and re-indexed metadata file
table_16S_bothPS = q2.Artifact.import_data('FeatureTable[Frequency]', table_16S_biom_bothPS)
md_round1and2_bothPS_df_16S_q2 = q2.Metadata(md_round1and2_bothPS_df_16S)

# Generate distance matrices using 'all_dists' utils
rare_depth_16S = 10000
dists_res_16S = all_dists(table_16S_bothPS,
                      rare_depth_16S, tree_16S)

# Generate ordinations (row=samples, cols=axes)
pcoa_res_16S = {}
pcoa_res_16S['Jaccard'] = pcoa(dists_res_16S['Jaccard'].distance_matrix).pcoa.view(OrdinationResults).samples
pcoa_res_16S['Unweighted UniFrac'] = pcoa(dists_res_16S['Unweighted UniFrac'].distance_matrix).pcoa.view(OrdinationResults).samples
pcoa_res_16S['Weighted UniFrac'] = pcoa(dists_res_16S['Weighted UniFrac'].distance_matrix).pcoa.view(OrdinationResults).samples
pcoa_res_16S['RPCA'] = dists_res_16S['RPCA'].biplot.view(OrdinationResults).samples


In [13]:
# Perform stepwise RDA-ANOVA
es_all = {}
use_ = ['round','sample_type','sample_type_2','sample_type_3','biomass_sample','extraction_kit_round']

# Clean up meta (only stuff to run)
mf_ord = md_round1and2_bothPS_df_16S_q2.to_dataframe().copy()

# Filter data
keep_ = [v_ for v_ in mf_ord.columns
         if len(set(mf_ord[v_])) > 1 and
         len(set(mf_ord[v_])) < mf_ord.shape[0]//2]
mf_ord = mf_ord[keep_]

# Run stepwise ANOVA for all RDA ordinations
for metric_, ord_ in  pcoa_res_16S.items():
    # get first three axes
    ord_ = ord_[[0,1,2]]
    ord_.columns = ['PC1','PC2','PC3']
    # subset/match
    mf_ord_ = mf_ord.copy()
    shared_ids = list(set(ord_.index)\
                      & set(mf_ord_.index))
    mf_ord_ = mf_ord_.loc[shared_ids,:]
    ord_ = ord_.loc[shared_ids,:]
    es_all[metric_] = run_stepwise_anova(ord_, mf_ord_, use_) #mf_ord_.columns)

# Concat output from all runs and export
es_alldf = pd.concat(es_all).rename({'+ sample_type_2':'Sample Type'}, axis=0)
es_alldf.to_csv('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/results/stepwise_anova/stepwise_anova_16S.txt', sep='\t')
es_alldf


Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: /Users/Justin/Mycelium/ipynb/assets/stepwise-rda.R /var/folders/jd/pqrf4k_j2ps8404x4rfp8_500000gn/T/tmp4wdmf1vc/ord_.tsv /var/folders/jd/pqrf4k_j2ps8404x4rfp8_500000gn/T/tmp4wdmf1vc/mf_.txt /var/folders/jd/pqrf4k_j2ps8404x4rfp8_500000gn/T/tmp4wdmf1vc/output.effect.size.tsv

R version 4.0.3 (2020-10-10) 


Loading required package: permute
Loading required package: lattice
This is vegan 2.5-7


Call: rda(formula = Y_16S ~ 1, data = X_16S, scale = TRUE)

              Inertia Rank
Total               3     
Unconstrained       3    3
Inertia is correlations 

Eigenvalues for unconstrained axes:
PC1 PC2 PC3 
  1   1   1 

Call: rda(formula = Y_16S ~ round + sample_type + sample_type_2 +
sample_type_3 + biomass_sample + extraction_kit_round, data = X_16S,
scale = TRUE)

              Inertia Proportion Rank
Total         3.00000    1.00000     
Constrained   2.88648    0.96216    3
Unconstrained 0.11352    0.03784    3
Inertia is correlations 
Some constraints were aliased because they were collinear (redundant)

Eigenvalues for constrained axes:
  RDA1   RDA2   RDA3 
0.9802 0.9721 0.9343 

Eigenvalues for unconstrained axes:
    PC1     PC2     PC3 
0.06575 0.02794 0.01984 

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary f

Loading required package: permute
Loading required package: lattice
This is vegan 2.5-7


Call: rda(formula = Y_16S ~ 1, data = X_16S, scale = TRUE)

              Inertia Rank
Total               3     
Unconstrained       3    3
Inertia is correlations 

Eigenvalues for unconstrained axes:
PC1 PC2 PC3 
  1   1   1 

Call: rda(formula = Y_16S ~ round + sample_type + sample_type_2 +
sample_type_3 + biomass_sample + extraction_kit_round, data = X_16S,
scale = TRUE)

              Inertia Proportion Rank
Total           3.000      1.000     
Constrained     2.832      0.944    3
Unconstrained   0.168      0.056    3
Inertia is correlations 
Some constraints were aliased because they were collinear (redundant)

Eigenvalues for constrained axes:
  RDA1   RDA2   RDA3 
0.9771 0.9329 0.9219 

Eigenvalues for unconstrained axes:
    PC1     PC2     PC3 
0.07808 0.06706 0.02287 

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary f

Loading required package: permute
Loading required package: lattice
This is vegan 2.5-7


Call: rda(formula = Y_16S ~ 1, data = X_16S, scale = TRUE)

              Inertia Rank
Total               3     
Unconstrained       3    3
Inertia is correlations 

Eigenvalues for unconstrained axes:
PC1 PC2 PC3 
  1   1   1 

Call: rda(formula = Y_16S ~ round + sample_type + sample_type_2 +
sample_type_3 + biomass_sample + extraction_kit_round, data = X_16S,
scale = TRUE)

              Inertia Proportion Rank
Total          3.0000     1.0000     
Constrained    2.6474     0.8825    3
Unconstrained  0.3526     0.1175    3
Inertia is correlations 
Some constraints were aliased because they were collinear (redundant)

Eigenvalues for constrained axes:
  RDA1   RDA2   RDA3 
0.9605 0.9274 0.7595 

Eigenvalues for unconstrained axes:
    PC1     PC2     PC3 
0.24055 0.07257 0.03952 

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary f

Loading required package: permute
Loading required package: lattice
This is vegan 2.5-7


Call: rda(formula = Y_16S ~ 1, data = X_16S, scale = TRUE)

              Inertia Rank
Total               3     
Unconstrained       3    3
Inertia is correlations 

Eigenvalues for unconstrained axes:
PC1 PC2 PC3 
  1   1   1 

Call: rda(formula = Y_16S ~ round + sample_type + sample_type_2 +
sample_type_3 + biomass_sample + extraction_kit_round, data = X_16S,
scale = TRUE)

              Inertia Proportion Rank
Total         3.00000    1.00000     
Constrained   2.71206    0.90402    3
Unconstrained 0.28794    0.09598    3
Inertia is correlations 
Some constraints were aliased because they were collinear (redundant)

Eigenvalues for constrained axes:
  RDA1   RDA2   RDA3 
0.9336 0.9305 0.8480 

Eigenvalues for unconstrained axes:
    PC1     PC2     PC3 
0.15204 0.06947 0.06643 



R2.adj  Df          AIC  \
Jaccard            + sample_type_3         0.955968  60 -1237.612400   
                   + extraction_kit_round  0.001832   6 -1259.471981   
Unweighted UniFrac + sample_type_3         0.935939  60  -997.663672   
                   + extraction_kit_round  0.001607   6 -1008.589714   
Weighted UniFrac   + sample_type_3         0.843865  60  -427.497609   
                   + extraction_kit_round  0.025052   6  -534.094338   
RPCA               + sample_type_3         0.888832  60  -644.890708   
                   + extraction_kit_round  0.004133   6  -663.805165   

                                                    F  Pr(>F)  
Jaccard            + sample_type_3         232.220113  0.0002  
                   + extraction_kit_round    5.188525  0.0002  
Unweighted UniFrac + sample_type_3         156.598053  0.0002  
                   + extraction_kit_round    3.483007  0.0002  
Weighted UniFrac   + sample_type_3          58.560362  0.0002  
                   + extraction_kit_round   19.442688  0.0002  
RPCA               + sample_type_3          86.151009  0.0002  
                   + extraction_kit_round    4.726246  0.0002

## Shotgun data

In [14]:
# Import data
md_round1and2_bothPS_q2 = q2.Metadata.load('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/sample_metadata/01_paired_files/12201_metadata_round1and2_bothPS.txt')

table_shotgun = q2.Artifact.load('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/shotgun/03_filtered_data/dna_bothPS_shotgun_woltka_wol_biom_noNTCs_noMock.qza')

tree_shotgun = q2.Artifact.load('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/shotgun/wol_tree.qza')


In [15]:
# Filter table
table_shotgun_biom = table_shotgun.view(Table)
md_round1and2_bothPS_df = md_round1and2_bothPS_q2.to_dataframe()
shared_ = list(set(table_shotgun_biom.ids()) & set(md_round1and2_bothPS_df.index))
md_round1and2_bothPS_df_shotgun = md_round1and2_bothPS_df.reindex(shared_)
table_shotgun_biom_bothPS = table_shotgun_biom.filter(shared_)
keep_ = table_shotgun_biom_bothPS.ids('observation')[table_shotgun_biom_bothPS.sum('observation') > 0]
table_shotgun_biom_bothPS.filter(keep_, axis='observation')

# Import filtered table and re-indexed metadata file
table_shotgun_bothPS = q2.Artifact.import_data('FeatureTable[Frequency]', table_shotgun_biom_bothPS)
md_round1and2_bothPS_df_shotgun_q2 = q2.Metadata(md_round1and2_bothPS_df_shotgun)

# Generate distance matrices using 'all_dissts' utils
rare_depth_shotgun = 2100
dists_res_shotgun = all_dists(table_shotgun_bothPS,
                      rare_depth_shotgun, tree_shotgun)

# Generate ordinations (row=samples, cols=axes)
pcoa_res_shotgun = {}
pcoa_res_shotgun['Jaccard'] = pcoa(dists_res_shotgun['Jaccard'].distance_matrix).pcoa.view(OrdinationResults).samples
pcoa_res_shotgun['Unweighted UniFrac'] = pcoa(dists_res_shotgun['Unweighted UniFrac'].distance_matrix).pcoa.view(OrdinationResults).samples
pcoa_res_shotgun['Weighted UniFrac'] = pcoa(dists_res_shotgun['Weighted UniFrac'].distance_matrix).pcoa.view(OrdinationResults).samples
pcoa_res_shotgun['RPCA'] = dists_res_shotgun['RPCA'].biplot.view(OrdinationResults).samples


In [16]:
# Perform stepwise RDA-ANOVA
es_all = {}
use_ = ['round','sample_type','sample_type_2','sample_type_3','biomass_sample','extraction_kit_round']

# Clean up meta (only stuff to run)
mf_ord = md_round1and2_bothPS_df_shotgun_q2.to_dataframe().copy()

# Filter data
keep_ = [v_ for v_ in mf_ord.columns
         if len(set(mf_ord[v_])) > 1 and
         len(set(mf_ord[v_])) < mf_ord.shape[0]//2]
mf_ord = mf_ord[keep_]

# Run stepwise ANOVA for all RDA ordinations
for metric_, ord_ in  pcoa_res_shotgun.items():
    # get first three axes
    ord_ = ord_[[0,1,2]]
    ord_.columns = ['PC1','PC2','PC3']
    # subset/match
    mf_ord_ = mf_ord.copy()
    shared_ids = list(set(ord_.index)\
                      & set(mf_ord_.index))
    mf_ord_ = mf_ord_.loc[shared_ids,:]
    ord_ = ord_.loc[shared_ids,:]
    es_all[metric_] = run_stepwise_anova(ord_, mf_ord_, use_) #mf_ord_.columns)

# Concat output from all runs and export
es_alldf = pd.concat(es_all).rename({'+ sample_type_2':'Sample Type'}, axis=0)
es_alldf.to_csv('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/results/stepwise_anova/stepwise_anova_shotgun.txt', sep='\t')
es_alldf


Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: /Users/Justin/Mycelium/ipynb/assets/stepwise-rda.R /var/folders/jd/pqrf4k_j2ps8404x4rfp8_500000gn/T/tmpz6fxttay/ord_.tsv /var/folders/jd/pqrf4k_j2ps8404x4rfp8_500000gn/T/tmpz6fxttay/mf_.txt /var/folders/jd/pqrf4k_j2ps8404x4rfp8_500000gn/T/tmpz6fxttay/output.effect.size.tsv

R version 4.0.3 (2020-10-10) 


Loading required package: permute
Loading required package: lattice
This is vegan 2.5-7


Call: rda(formula = Y_16S ~ 1, data = X_16S, scale = TRUE)

              Inertia Rank
Total               3     
Unconstrained       3    3
Inertia is correlations 

Eigenvalues for unconstrained axes:
PC1 PC2 PC3 
  1   1   1 

Call: rda(formula = Y_16S ~ round + sample_type + sample_type_2 +
sample_type_3 + biomass_sample + extraction_kit_round, data = X_16S,
scale = TRUE)

              Inertia Proportion Rank
Total         3.00000    1.00000     
Constrained   2.82614    0.94205    3
Unconstrained 0.17386    0.05795    3
Inertia is correlations 
Some constraints were aliased because they were collinear (redundant)

Eigenvalues for constrained axes:
  RDA1   RDA2   RDA3 
0.9856 0.9643 0.8762 

Eigenvalues for unconstrained axes:
    PC1     PC2     PC3 
0.12380 0.03571 0.01435 

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary f

Loading required package: permute
Loading required package: lattice
This is vegan 2.5-7


Call: rda(formula = Y_16S ~ 1, data = X_16S, scale = TRUE)

              Inertia Rank
Total               3     
Unconstrained       3    3
Inertia is correlations 

Eigenvalues for unconstrained axes:
PC1 PC2 PC3 
  1   1   1 

Call: rda(formula = Y_16S ~ round + sample_type + sample_type_2 +
sample_type_3 + biomass_sample + extraction_kit_round, data = X_16S,
scale = TRUE)

              Inertia Proportion Rank
Total         3.00000    1.00000     
Constrained   2.79913    0.93304    3
Unconstrained 0.20087    0.06696    3
Inertia is correlations 
Some constraints were aliased because they were collinear (redundant)

Eigenvalues for constrained axes:
  RDA1   RDA2   RDA3 
0.9701 0.9555 0.8736 

Eigenvalues for unconstrained axes:
    PC1     PC2     PC3 
0.12642 0.04454 0.02991 

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary f

Loading required package: permute
Loading required package: lattice
This is vegan 2.5-7


Call: rda(formula = Y_16S ~ 1, data = X_16S, scale = TRUE)

              Inertia Rank
Total               3     
Unconstrained       3    3
Inertia is correlations 

Eigenvalues for unconstrained axes:
PC1 PC2 PC3 
  1   1   1 

Call: rda(formula = Y_16S ~ round + sample_type + sample_type_2 +
sample_type_3 + biomass_sample + extraction_kit_round, data = X_16S,
scale = TRUE)

              Inertia Proportion Rank
Total         3.00000    1.00000     
Constrained   2.70095    0.90032    3
Unconstrained 0.29905    0.09968    3
Inertia is correlations 
Some constraints were aliased because they were collinear (redundant)

Eigenvalues for constrained axes:
  RDA1   RDA2   RDA3 
0.9103 0.8980 0.8926 

Eigenvalues for unconstrained axes:
    PC1     PC2     PC3 
0.10737 0.10199 0.08969 

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary f

Loading required package: permute
Loading required package: lattice
This is vegan 2.5-7


Call: rda(formula = Y_16S ~ 1, data = X_16S, scale = TRUE)

              Inertia Rank
Total               3     
Unconstrained       3    3
Inertia is correlations 

Eigenvalues for unconstrained axes:
PC1 PC2 PC3 
  1   1   1 

Call: rda(formula = Y_16S ~ round + sample_type + sample_type_2 +
sample_type_3 + biomass_sample + extraction_kit_round, data = X_16S,
scale = TRUE)

              Inertia Proportion Rank
Total          3.0000     1.0000     
Constrained    2.6284     0.8761    3
Unconstrained  0.3716     0.1239    3
Inertia is correlations 
Some constraints were aliased because they were collinear (redundant)

Eigenvalues for constrained axes:
  RDA1   RDA2   RDA3 
0.9287 0.8753 0.8244 

Eigenvalues for unconstrained axes:
    PC1     PC2     PC3 
0.17560 0.12466 0.07135 



R2.adj  Df          AIC  \
Jaccard            + sample_type_3         0.933308  65 -1198.449457   
                   + extraction_kit_round  0.002959   6 -1228.614727   
Unweighted UniFrac + sample_type_3         0.921696  65 -1072.603252   
                   + extraction_kit_round  0.004670   6 -1115.388030   
Weighted UniFrac   + sample_type_3         0.868938  65  -668.788820   
                   + extraction_kit_round  0.021436   6  -803.389366   
RPCA               + sample_type_3         0.853082  65  -579.250730   
                   + extraction_kit_round  0.010697   6  -633.096676   

                                                    F  Pr(>F)  
Jaccard            + sample_type_3         169.578316  0.0002  
                   + extraction_kit_round    6.556900  0.0002  
Unweighted UniFrac + sample_type_3         142.792139  0.0002  
                   + extraction_kit_round    8.588847  0.0002  
Weighted UniFrac   + sample_type_3          80.865936  0.0002  
                   + extraction_kit_round   24.399744  0.0002  
RPCA               + sample_type_3          70.946186  0.0002  
                   + extraction_kit_round   10.396974  0.0002

## ITS data

In [17]:
# Import data
md_round1and2_bothPS_q2 = q2.Metadata.load('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/sample_metadata/01_paired_files/12201_metadata_round1and2_bothPS.txt')

table_its = q2.Artifact.load('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/data/ITS/08_filtered_data/dna_bothPS_ITS_deblur_biom_lod_noNTCs_noMock.qza')


In [18]:
# Filter table
table_its_biom = table_its.view(Table)
md_round1and2_bothPS_df = md_round1and2_bothPS_q2.to_dataframe()
shared_ = list(set(table_its_biom.ids()) & set(md_round1and2_bothPS_df.index))
md_round1and2_bothPS_df_its = md_round1and2_bothPS_df.reindex(shared_)
table_its_biom_bothPS = table_its_biom.filter(shared_)
keep_ = table_its_biom_bothPS.ids('observation')[table_its_biom_bothPS.sum('observation') > 0]
table_its_biom_bothPS.filter(keep_, axis='observation')

# Import filtered table and re-indexed metadata file
table_its_bothPS = q2.Artifact.import_data('FeatureTable[Frequency]', table_its_biom_bothPS)
md_round1and2_bothPS_df_its_q2 = q2.Metadata(md_round1and2_bothPS_df_its)

# Generate distance matrices using 'all_dissts' utils
rare_depth_its = 630
dists_res_its = all_dists_no_tree(table_its_bothPS,
                      rare_depth_its)

# Generate ordinations (row=samples, cols=axes)
pcoa_res_its = {}
pcoa_res_its['Jaccard'] = pcoa(dists_res_its['Jaccard'].distance_matrix).pcoa.view(OrdinationResults).samples
pcoa_res_its['RPCA'] = dists_res_its['RPCA'].biplot.view(OrdinationResults).samples


In [19]:
# Perform stepwise RDA-ANOVA
es_all = {}
use_ = ['round','sample_type','sample_type_2','sample_type_3','biomass_sample','extraction_kit_round']

# Clean up meta (only stuff to run)
mf_ord = md_round1and2_bothPS_df_its_q2.to_dataframe().copy()

# Filter data
keep_ = [v_ for v_ in mf_ord.columns
         if len(set(mf_ord[v_])) > 1 and
         len(set(mf_ord[v_])) < mf_ord.shape[0]//2]
mf_ord = mf_ord[keep_]

# Run stepwise ANOVA for all RDA ordinations
for metric_, ord_ in  pcoa_res_its.items():
    # get first three axes
    ord_ = ord_[[0,1,2]]
    ord_.columns = ['PC1','PC2','PC3']
    # subset/match
    mf_ord_ = mf_ord.copy()
    shared_ids = list(set(ord_.index)\
                      & set(mf_ord_.index))
    mf_ord_ = mf_ord_.loc[shared_ids,:]
    ord_ = ord_.loc[shared_ids,:]
    es_all[metric_] = run_stepwise_anova(ord_, mf_ord_, use_) #mf_ord_.columns)

# Concat output from all runs and export
es_alldf = pd.concat(es_all).rename({'+ sample_type_2':'Sample Type'}, axis=0)
es_alldf.to_csv('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/results/stepwise_anova/stepwise_anova_its.txt', sep='\t')
es_alldf


Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: /Users/Justin/Mycelium/ipynb/assets/stepwise-rda.R /var/folders/jd/pqrf4k_j2ps8404x4rfp8_500000gn/T/tmpz7_toh5k/ord_.tsv /var/folders/jd/pqrf4k_j2ps8404x4rfp8_500000gn/T/tmpz7_toh5k/mf_.txt /var/folders/jd/pqrf4k_j2ps8404x4rfp8_500000gn/T/tmpz7_toh5k/output.effect.size.tsv

R version 4.0.3 (2020-10-10) 


Loading required package: permute
Loading required package: lattice
This is vegan 2.5-7


Call: rda(formula = Y_16S ~ 1, data = X_16S, scale = TRUE)

              Inertia Rank
Total               3     
Unconstrained       3    3
Inertia is correlations 

Eigenvalues for unconstrained axes:
PC1 PC2 PC3 
  1   1   1 

Call: rda(formula = Y_16S ~ round + sample_type + sample_type_2 +
sample_type_3 + biomass_sample + extraction_kit_round, data = X_16S,
scale = TRUE)

              Inertia Proportion Rank
Total          3.0000     1.0000     
Constrained    2.5312     0.8437    3
Unconstrained  0.4688     0.1563    3
Inertia is correlations 
Some constraints were aliased because they were collinear (redundant)

Eigenvalues for constrained axes:
  RDA1   RDA2   RDA3 
0.9447 0.8944 0.6922 

Eigenvalues for unconstrained axes:
    PC1     PC2     PC3 
0.30780 0.10561 0.05535 

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary f

Loading required package: permute
Loading required package: lattice
This is vegan 2.5-7


Call: rda(formula = Y_16S ~ 1, data = X_16S, scale = TRUE)

              Inertia Rank
Total               3     
Unconstrained       3    3
Inertia is correlations 

Eigenvalues for unconstrained axes:
PC1 PC2 PC3 
  1   1   1 

Call: rda(formula = Y_16S ~ round + sample_type + sample_type_2 +
sample_type_3 + biomass_sample + extraction_kit_round, data = X_16S,
scale = TRUE)

              Inertia Proportion Rank
Total          3.0000     1.0000     
Constrained    2.3262     0.7754    3
Unconstrained  0.6738     0.2246    3
Inertia is correlations 
Some constraints were aliased because they were collinear (redundant)

Eigenvalues for constrained axes:
  RDA1   RDA2   RDA3 
0.8449 0.7824 0.6989 

Eigenvalues for unconstrained axes:
    PC1     PC2     PC3 
0.30111 0.21761 0.15508 



R2.adj  Df         AIC          F  Pr(>F)
Jaccard + sample_type_3         0.795789  59 -183.617659  33.231961  0.0002
        + extraction_kit_round  0.023947   6 -239.497438  10.498138  0.0002
RPCA    + sample_type_3         0.720444  59  -30.046851  22.315633  0.0002
        + extraction_kit_round  0.020443   6  -62.068822   6.641175  0.0002